In [4]:
import pandas as pd
import numpy as np
import datetime
import io

# Cleaning up imported data

This notebook show some common things that you need to do when working with dataframes.

## Inspectinga DataFrame

There are several methods to inspect a dataframe and it is good to know there.

### Describe

The describe() method will give you the mean and quartile values for all numeric columns.

In [3]:
csv_data = """
Start,Finish,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count)
01-01-2014 12:34:00,02-01-2014,0.0,0.0,11.0942744372862,0.0,0.0,22658.0
02-01-2014 10:56:00,03-01-2014,0.0,0.0,6.29971590909092,0.0,0.0,12866.0
03-01-2014 08:01:00,04-01-2014,0.0,0.0,9.91326279527559,0.0,0.0,
04-01-2014 01:02:00,05-01-2014,0.0,0.0,11.0771370198043,0.0,0.0,22623.0
"""
df = pd.read_csv(io.StringIO(csv_data))
df.describe()

,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count)
count,4.0,4.0,4.000000,4.0,4.0,3.000000
mean,0.0,0.0,9.596098,0.0,0.0,19382.333333
std,0.0,0.0,2.266035,0.0,0.0,5643.337340
min,0.0,0.0,6.299716,0.0,0.0,12866.000000
25%,0.0,0.0,9.009876,0.0,0.0,17744.500000
50%,0.0,0.0,10.495200,0.0,0.0,22623.000000
75%,0.0,0.0,11.081421,0.0,0.0,22640.500000
max,0.0,0.0,11.094274,0.0,0.0,22658.000000


### Info

The info() method will display the datatypes of the columns as well as how many values are in them.  If you only want the data types you can just use the dtypes attribute.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
Start                      4 non-null object
Finish                     4 non-null object
Active Calories (kcal)     4 non-null float64
Cycling Distance (mi)      4 non-null float64
Distance (mi)              4 non-null float64
Flights Climbed (count)    4 non-null float64
Resting Calories (kcal)    4 non-null float64
Steps (count)              3 non-null float64
dtypes: float64(6), object(2)
memory usage: 336.0+ bytes


In [8]:
df.dtypes

Start                       object
Finish                      object
Active Calories (kcal)     float64
Cycling Distance (mi)      float64
Distance (mi)              float64
Flights Climbed (count)    float64
Resting Calories (kcal)    float64
Steps (count)              float64
dtype: object

## Type Conversions

### Converting strings to datetime objects

We have 2 different date formats here, with hours/minutes and without. Since this was read from CSV these are just parsed strings and not datetime and date objects as we would like so we will convert them.

In [88]:
df['Start_dt'] = pd.to_datetime(df['Start'])
df['Finish_dt'] = pd.to_datetime(df['Finish'])  # intelligently infers format
df.drop(['Start', 'Finish'], axis=1, inplace=True)
df

,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count),Start_dt,Finish_dt
0,0.0,0.0,11.094274,0.0,0.0,22658.0,2014-01-01 12:34:00,2014-02-01
1,0.0,0.0,6.299716,0.0,0.0,12866.0,2014-02-01 10:56:00,2014-03-01
2,0.0,0.0,9.913263,0.0,0.0,NaN,2014-03-01 08:01:00,2014-04-01
3,0.0,0.0,11.077137,0.0,0.0,22623.0,2014-04-01 01:02:00,2014-05-01


### Decomposing dates

Often we want columns that contains subsets of the dates such as years or quarters. When you have a Series of datetime objects you cannot access the datetime attributes directly but have to use the **dt** accessor.

In [89]:
df['year'] = df.Start_dt.dt.year
df

,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count),Start_dt,Finish_dt,year
0,0.0,0.0,11.094274,0.0,0.0,22658.0,2014-01-01 12:34:00,2014-02-01,2014
1,0.0,0.0,6.299716,0.0,0.0,12866.0,2014-02-01 10:56:00,2014-03-01,2014
2,0.0,0.0,9.913263,0.0,0.0,NaN,2014-03-01 08:01:00,2014-04-01,2014
3,0.0,0.0,11.077137,0.0,0.0,22623.0,2014-04-01 01:02:00,2014-05-01,2014


There is no quarter accessor nn a datetime so we'll compute the quarter.  In this case we will use the **apply** method to create a lambda function which will get applied to each cell in the Series.

In [90]:
df['quarter'] = df.Start_dt.apply(lambda x: x.year + int(x.month/3+1)/10)
df

,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count),Start_dt,Finish_dt,year,quarter
0,0.0,0.0,11.094274,0.0,0.0,22658.0,2014-01-01 12:34:00,2014-02-01,2014,2014.1
1,0.0,0.0,6.299716,0.0,0.0,12866.0,2014-02-01 10:56:00,2014-03-01,2014,2014.1
2,0.0,0.0,9.913263,0.0,0.0,NaN,2014-03-01 08:01:00,2014-04-01,2014,2014.2
3,0.0,0.0,11.077137,0.0,0.0,22623.0,2014-04-01 01:02:00,2014-05-01,2014,2014.2


### Handling Missing Data (NaN)

When data is missing, we will end up with NaNs in our dataframe. Sometimes this is ok but it will prevent some numeric functions from running so often we want to remove them. There are a few ways to do this.

We can just drop the rows with NaNs

In [91]:
df.dropna()

,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count),Start_dt,Finish_dt,year,quarter
0,0.0,0.0,11.094274,0.0,0.0,22658.0,2014-01-01 12:34:00,2014-02-01,2014,2014.1
1,0.0,0.0,6.299716,0.0,0.0,12866.0,2014-02-01 10:56:00,2014-03-01,2014,2014.1
3,0.0,0.0,11.077137,0.0,0.0,22623.0,2014-04-01 01:02:00,2014-05-01,2014,2014.2


Or we can replace the NaNs with a numeric value

In [92]:
df = df.fillna(0)
df

,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count),Start_dt,Finish_dt,year,quarter
0,0.0,0.0,11.094274,0.0,0.0,22658.0,2014-01-01 12:34:00,2014-02-01,2014,2014.1
1,0.0,0.0,6.299716,0.0,0.0,12866.0,2014-02-01 10:56:00,2014-03-01,2014,2014.1
2,0.0,0.0,9.913263,0.0,0.0,0.0,2014-03-01 08:01:00,2014-04-01,2014,2014.2
3,0.0,0.0,11.077137,0.0,0.0,22623.0,2014-04-01 01:02:00,2014-05-01,2014,2014.2


### Integers, floats and rounding

The numeric types we get in our dataframe may not be what we want and these can be coverted.

In [93]:
# let's convert the steps to an integer
df['Steps (count)'] = df['Steps (count)'].astype(int)

# and round the distance to two decimal places
df['Distance (mi)'] = df['Distance (mi)'].round(2)
df

,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count),Start_dt,Finish_dt,year,quarter
0,0.0,0.0,11.09,0.0,0.0,22658,2014-01-01 12:34:00,2014-02-01,2014,2014.1
1,0.0,0.0,6.30,0.0,0.0,12866,2014-02-01 10:56:00,2014-03-01,2014,2014.1
2,0.0,0.0,9.91,0.0,0.0,0,2014-03-01 08:01:00,2014-04-01,2014,2014.2
3,0.0,0.0,11.08,0.0,0.0,22623,2014-04-01 01:02:00,2014-05-01,2014,2014.2


## Renaming and reordering columns

Often the names we get for columns are not what we want.  It is easier to work with columns that don't have spaces.  We can simply replace the spaces with underscores.

In [82]:
df.rename(columns=lambda x: x.replace(" ", "_"), inplace=True)
df

,Active_Calories_(kcal),Cycling_Distance_(mi),Distance_(mi),Flights_Climbed_(count),Resting_Calories_(kcal),Steps_(count),Start_dt,Finish_dt,year,quarter
0,0.0,0.0,11.094274,0.0,0.0,22658.0,2014-01-01 12:34:00,2014-02-01,2014,2014.1
1,0.0,0.0,6.299716,0.0,0.0,12866.0,2014-02-01 10:56:00,2014-03-01,2014,2014.1
2,0.0,0.0,9.913263,0.0,0.0,NaN,2014-03-01 08:01:00,2014-04-01,2014,2014.2
3,0.0,0.0,11.077137,0.0,0.0,22623.0,2014-04-01 01:02:00,2014-05-01,2014,2014.2


This works well for code based transformations but often we want to select the names individually.  We can create a dictionary to map names

In [83]:
m = {"Active_Calories_(kcal)": 'actcal', 'Cycling_Distance_(mi)': 'cycling', 'Distance_(mi)': 'walkdist',
     "Flights_Climbed_(count)": 'flights', 'Resting_Calories_(kcal)': 'restcals'}
df.rename(columns=m)

,actcal,cycling,walkdist,flights,restcals,Steps_(count),Start_dt,Finish_dt,year,quarter
0,0.0,0.0,11.094274,0.0,0.0,22658.0,2014-01-01 12:34:00,2014-02-01,2014,2014.1
1,0.0,0.0,6.299716,0.0,0.0,12866.0,2014-02-01 10:56:00,2014-03-01,2014,2014.1
2,0.0,0.0,9.913263,0.0,0.0,NaN,2014-03-01 08:01:00,2014-04-01,2014,2014.2
3,0.0,0.0,11.077137,0.0,0.0,22623.0,2014-04-01 01:02:00,2014-05-01,2014,2014.2


If we want to place most of the column names though it's simple to just set them directly.

In [84]:
df.columns = ['actcal', 'cycling', 'walkdist', 'flights', 'restcals', 'steps', 'start', 'finish', 'year', 'quarter']
df

,actcal,cycling,walkdist,flights,restcals,steps,start,finish,year,quarter
0,0.0,0.0,11.094274,0.0,0.0,22658.0,2014-01-01 12:34:00,2014-02-01,2014,2014.1
1,0.0,0.0,6.299716,0.0,0.0,12866.0,2014-02-01 10:56:00,2014-03-01,2014,2014.1
2,0.0,0.0,9.913263,0.0,0.0,NaN,2014-03-01 08:01:00,2014-04-01,2014,2014.2
3,0.0,0.0,11.077137,0.0,0.0,22623.0,2014-04-01 01:02:00,2014-05-01,2014,2014.2


When looking at this I'd like to hav ethe dates first so we will re-order the columns

In [85]:
df = df[['start', 'finish', 'year', 'quarter','actcal', 'cycling', 'walkdist', 'flights', 'restcals', 'steps']]
df

,start,finish,year,quarter,actcal,cycling,walkdist,flights,restcals,steps
0,2014-01-01 12:34:00,2014-02-01,2014,2014.1,0.0,0.0,11.094274,0.0,0.0,22658.0
1,2014-02-01 10:56:00,2014-03-01,2014,2014.1,0.0,0.0,6.299716,0.0,0.0,12866.0
2,2014-03-01 08:01:00,2014-04-01,2014,2014.2,0.0,0.0,9.913263,0.0,0.0,NaN
3,2014-04-01 01:02:00,2014-05-01,2014,2014.2,0.0,0.0,11.077137,0.0,0.0,22623.0


## Merging Dataframes

### Concatenating

Concatenating is adding the rows of one dataframe to another when the columns match.

In [4]:
df1 = pd.DataFrame( {'Rev': [20, 22, 23],
                     'Exp': [19, 19.5, 20]},
                    index=[2010, 2011, 2012])
df1

,Rev,Exp
2010,20,19.0
2011,22,19.5
2012,23,20.0


In [7]:
df2 = pd.DataFrame( {'Rev': [24, 25],
                     'Exp': [21, 22]},
                    index=[2013, 2014])
df2

,Rev,Exp
2013,24,21
2014,25,22


In [12]:
df12 = pd.concat([df1, df2])
df12

,Rev,Exp
2010,20,19.0
2011,22,19.5
2012,23,20.0
2013,24,21.0
2014,25,22.0


### Merging

In [10]:
df3 = pd.DataFrame( {'Budget': [20, 21, 22]},
                    index=[2010, 2012, 2014])
df3

,Budget
2010,20
2012,21
2014,22


We can add columns to the dataframe where the index matches.  In the simplest case, we end up with only the rows that match.

In [19]:
pd.merge(df12, df3, left_index=True, right_index=True)

,Rev,Exp,Budget
2010,20,19.0,20
2012,23,20.0,21
2014,25,22.0,22


But we can specify we want all the rows.

In [20]:
pd.merge(df12, df3, left_index=True, right_index=True, how='outer')

,Rev,Exp,Budget
2010,20,19.0,20.0
2011,22,19.5,NaN
2012,23,20.0,21.0
2013,24,21.0,NaN
2014,25,22.0,22.0


In [8]:
## Working with Indexes

In [6]:
csv_data = """
Start,Finish,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count)
01-01-2014 12:34:00,02-01-2014,0.0,0.0,11.0942744372862,0.0,0.0,22658.0
02-01-2014 10:56:00,03-01-2014,0.0,0.0,6.29971590909092,0.0,0.0,12866.0
03-01-2014 08:01:00,04-01-2014,0.0,0.0,9.91326279527559,0.0,0.0,
04-01-2014 01:02:00,05-01-2014,0.0,0.0,11.0771370198043,0.0,0.0,22623.0
01-01-2015 12:34:00,02-01-2014,0.0,0.0,11.0942744372862,0.0,0.0,22658.0
02-01-2015 10:56:00,03-01-2014,0.0,0.0,6.29971590909092,0.0,0.0,12866.0
03-01-2015 08:01:00,04-01-2014,0.0,0.0,9.91326279527559,0.0,0.0,
04-01-2015 01:02:00,05-01-2014,0.0,0.0,11.0771370198043,0.0,0.0,22623.0
"""
df = pd.read_csv(io.StringIO(csv_data))
df

,Start,Finish,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count)
0,01-01-2014 12:34:00,02-01-2014,0.0,0.0,11.094274,0.0,0.0,22658.0
1,02-01-2014 10:56:00,03-01-2014,0.0,0.0,6.299716,0.0,0.0,12866.0
2,03-01-2014 08:01:00,04-01-2014,0.0,0.0,9.913263,0.0,0.0,NaN
3,04-01-2014 01:02:00,05-01-2014,0.0,0.0,11.077137,0.0,0.0,22623.0
4,01-01-2015 12:34:00,02-01-2014,0.0,0.0,11.094274,0.0,0.0,22658.0
5,02-01-2015 10:56:00,03-01-2014,0.0,0.0,6.299716,0.0,0.0,12866.0
6,03-01-2015 08:01:00,04-01-2014,0.0,0.0,9.913263,0.0,0.0,NaN
7,04-01-2015 01:02:00,05-01-2014,0.0,0.0,11.077137,0.0,0.0,22623.0


In [9]:
# get the index
df.index

RangeIndex(start=0, stop=8, step=1)

In [10]:
# on a subset, the index will not be continuous
df1 = df[df['Distance (mi)'] > 10]
df1

,Start,Finish,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count)
0,01-01-2014 12:34:00,02-01-2014,0.0,0.0,11.094274,0.0,0.0,22658.0
3,04-01-2014 01:02:00,05-01-2014,0.0,0.0,11.077137,0.0,0.0,22623.0
4,01-01-2015 12:34:00,02-01-2014,0.0,0.0,11.094274,0.0,0.0,22658.0
7,04-01-2015 01:02:00,05-01-2014,0.0,0.0,11.077137,0.0,0.0,22623.0


In [11]:
# We can reset this to a continuous index
df_with_new_index = df1.reset_index()

,index,Start,Finish,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count)
0,0,01-01-2014 12:34:00,02-01-2014,0.0,0.0,11.094274,0.0,0.0,22658.0
1,3,04-01-2014 01:02:00,05-01-2014,0.0,0.0,11.077137,0.0,0.0,22623.0
2,4,01-01-2015 12:34:00,02-01-2014,0.0,0.0,11.094274,0.0,0.0,22658.0
3,7,04-01-2015 01:02:00,05-01-2014,0.0,0.0,11.077137,0.0,0.0,22623.0


In [13]:
# set the index to a specific column
df2 = df1.set_index('Start')
df2

,Finish,Active Calories (kcal),Cycling Distance (mi),Distance (mi),Flights Climbed (count),Resting Calories (kcal),Steps (count)
Start,,,,,,,
01-01-2014 12:34:00,02-01-2014,0.0,0.0,11.094274,0.0,0.0,22658.0
04-01-2014 01:02:00,05-01-2014,0.0,0.0,11.077137,0.0,0.0,22623.0
01-01-2015 12:34:00,02-01-2014,0.0,0.0,11.094274,0.0,0.0,22658.0
04-01-2015 01:02:00,05-01-2014,0.0,0.0,11.077137,0.0,0.0,22623.0
